# Update the values of a raster
- with a csv holding the current values and the ones to update
- I update value per value the raster in an array and update the raster
- WARNING: for this script for now, make a copy of the file

In [2]:
import os
import gdal
import geopandas as gpd
import numpy as np

In [2]:
raster_path = r"C:\Users\admin\Downloads\forest\ForestCondition_2018_Uncertainty.tif"
csv_path = r"C:\Users\admin\Downloads\forest\ForestCondition_2018_Uncertainty_reclass.csv"
output_raster_path = r"C:\Users\admin\Downloads\forest"

In [3]:
raster_dataset = gdal.Open(raster_path, gdal.GA_Update)  # Open the raster in update mode
# Check if the raster dataset was successfully opened
if raster_dataset is None:
    raise Exception("Failed to open the raster dataset.")

# Load the CSV data
csv_data = gpd.read_file(csv_path)

In [4]:
# Iterate through the CSV data and update the raster values
for index, row in csv_data.iterrows():
    current_value = float(row['Current_value'])  # Convert to float
    new_value = float(row['New_value'])  # Convert to float

    # Read the raster band as an array
    raster_band = raster_dataset.GetRasterBand(1)
    raster_array = raster_band.ReadAsArray()

    # Update the raster values where the current value matches
    raster_array[raster_array == current_value] = new_value

    # Write the updated array back to the raster band
    raster_band.WriteArray(raster_array, 0, 0)

# Close and save the dataset
raster_dataset = None 
# time: 45mins

# Update SOME raster values

In [1]:
# Input raster file path
input_raster_path = r"C:\Users\admin\Downloads\forest\ForestTypology_2018.tif"

# Output raster file path
output_raster_path = r"C:\Users\admin\Downloads\forest\ForestTypology_2018_fixed.tif"

In [3]:
# Define the reclassification mapping as a dictionary
reclass_mapping = {
    211: 65535,
    212: 65535, 
    213: 65535,
    224: 65535
}

# Open the input raster using GDAL
input_raster = gdal.Open(input_raster_path, gdal.GA_ReadOnly)
if input_raster is None:
    print("Could not open the input raster file.")
    exit(1)

# Read the raster data into a NumPy array
band = input_raster.GetRasterBand(1)
raster_data = band.ReadAsArray()



In [4]:
# Iterate through the reclassification mapping and apply changes
for current_value, new_value in reclass_mapping.items():
    raster_data[raster_data == current_value] = new_value

In [5]:
# Create a new output raster with the reclassified data
driver = gdal.GetDriverByName("GTiff")
output_raster = driver.Create(output_raster_path, input_raster.RasterXSize, input_raster.RasterYSize, 1, band.DataType)

# Write the reclassified data to the output raster
output_band = output_raster.GetRasterBand(1)
output_band.WriteArray(raster_data)

# Set the projection and geotransform
output_raster.SetProjection(input_raster.GetProjection())
output_raster.SetGeoTransform(input_raster.GetGeoTransform())

# Set the NoData value in the output raster's metadata
output_band.SetNoDataValue(65535)

# Close the raster datasets
input_raster = None
output_raster = None